In [ ]:
# Core ROS and robotics libraries
import rospy
import actionlib
import math
import time
import threading
#visulization or plo
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np



# ROS message types
from geometry_msgs.msg import Point, PoseStamped
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from assignment_2_2024.msg import PlanningAction, PlanningGoal

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display

# More reliable than notebook mode for basic plots
%matplotlib inline  

In [ ]:
# Initialize ROS node (only run this once)
try:
    rospy.init_node('Robot_interface', anonymous=True)
    rospy.loginfo("ROS node initialized")
except rospy.ROSException:
    rospy.logwarn("ROS node already exists")

# Verify ROS master connection
try:
    rospy.get_master().getPid()
    print("✅ Connected to ROS master")
except:
    print("❌ Failed to connect to ROS master")
    raise

In [ ]:
class ActionClient:
    def __init__(self):
        self.client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
        self.connected = False
        self.connect_to_server()
    
    def connect_to_server(self, timeout=10):
        """Wait for action server with visual feedback"""
        print("🔌 Connecting to action server...")
        for i in range(timeout):
            if self.client.wait_for_server(rospy.Duration(1.0)):
                self.connected = True
                print("✅ Connected to action server!")
                return True
            print(f"⏳ Attempt {i+1}/{timeout}...")
        print("❌ Failed to connect to action server")
        return False

# Test connection
action_client = ActionClient()

In [ ]:
#State Monitoring 

class RobotStateMonitor:
    def __init__(self):
        self.position = Point()
        self.min_distance = float('inf')
        
        # Setup subscribers
        rospy.Subscriber('/odom', Odometry, self.odom_callback)
        rospy.Subscriber('/scan', LaserScan, self.scan_callback)
        time.sleep(1)  # Allow time for subscriptions to establish
    
    def odom_callback(self, msg):
        """Update current robot position"""
        self.position = msg.pose.pose.position
    
    def scan_callback(self, msg):
        """Update minimum obstacle distance"""
        valid_ranges = [r for r in msg.ranges if not math.isinf(r) and r > 0.1]
        self.min_distance = min(valid_ranges) if valid_ranges else float('inf')

# Initialize monitor
state_monitor = RobotStateMonitor()
print("📊 Robot state monitor ready")

In [ ]:
class GoalTracker:
    def __init__(self):
        self.reached = 0
        self.cancelled = 0
        self.x_goals = []
        self.y_goals = []
        self.status = []  # True=reached, False=cancelled
    
    def add_goal(self, x, y):
        self.x_goals.append(x)
        self.y_goals.append(y)
        self.status.append(False)  # Initially not reached
    
    def mark_reached(self):
        if self.status:
            self.status[-1] = True
            self.reached += 1
    
    def mark_cancelled(self):
        self.cancelled += 1

tracker = GoalTracker()

In [ ]:
# Input fields
x_input = widgets.FloatText(description="Target X:")
y_input = widgets.FloatText(description="Target Y:")

# Control buttons
set_btn = widgets.Button(description="Set Goal", button_style='success')
cancel_btn = widgets.Button(description="Cancel Goal", button_style='danger')

# Status displays
position_label = widgets.Label()
distance_label = widgets.Label()
status_output = widgets.Output()

In [ ]:
def set_goal(_):
    global current_goal
    
    goal = PlanningGoal()
    goal.target_pose = PoseStamped()
    goal.target_pose.pose.position.x = x_input.value
    goal.target_pose.pose.position.y = y_input.value
    
    # Track new goal
    current_goal = [x_input.value, y_input.value]
    tracker.add_goal(x_input.value, y_input.value)
    
    # Send and update
    action_client.client.send_goal(goal, done_cb=goal_result_callback)
    with status_output:
        print(f"Moving to: ({x_input.value}, {y_input.value})")
    
    # Start live updates
    while not rospy.is_shutdown() and current_goal[0] is not None:
        update_plots()
        rospy.sleep(0.1)

def cancel_goal(_):
    global current_goal
    action_client.client.cancel_goal()
    current_goal = [None, None]
    tracker.mark_cancelled()
    with status_output:
        print("Goal cancelled")
    update_plots()

def goal_result_callback(status, result):
    global current_goal
    current_goal = [None, None]
    if status == actionlib.GoalStatus.SUCCEEDED:
        tracker.mark_reached()
        with status_output:
            print(f"Goal reached! Final position: ({result.actual_pose.position.x:.2f}, {result.actual_pose.position.y:.2f})")
    update_plots()
# Connect buttons
set_btn.on_click(set_goal)
cancel_btn.on_click(cancel_goal)

In [ ]:

# Create persistent figure object
plt.close('all')
fig, (ax_pos, ax_stats) = plt.subplots(1, 2, figsize=(12,5))

# Position plot setup
ax_pos.set_title('Live Robot Movement')
ax_pos.set_xlabel('X [m]')
ax_pos.set_ylabel('Y [m]')
ax_pos.grid(True)

# Elements we'll update
path_line, = ax_pos.plot([], [], 'b-', alpha=0.5, label='Path')
current_pos = ax_pos.plot([], [], 'ro', markersize=8, label='Current')[0]
goal_point = ax_pos.plot([], [], 'g*', markersize=10, label='Target')[0]
ax_pos.legend()

# Stats plot setup
status_bars = ax_stats.bar(['Reached', 'Cancelled'], [0, 0], color=['green', 'red'])
ax_stats.set_title('Goal Completion')
ax_stats.set_ylabel('Count')

plt.tight_layout()
plt.show()